In [ ]:
import pandas as pd

In [ ]:
### Loop the data lines
with open("/content/news_text_25_5.csv", 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
# column_names = ['Link','Title', 'Date','Content']

### Read csv
df = pd.read_csv("/content/news_text_25_5.csv", delimiter=",")
# df.drop(index=df.index[0], axis=0, inplace=True)
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# dropna row
# get a cell value
index = 1
col = "text"
cell_val = df.iloc[index][col]
print ("Cell value at", index, "for column", col, ":", cell_val)

In [ ]:
with open('test.txt', 'w') as f:
  f.write(cell_val)
  f.close()

In [ ]:
!pip install pyvi

Hàm chuẩn hóa văn bản tiếng Việt với 5 bước
- Chuẩn hóa unicode 
- Chuẩn hóa dấu câu tiếng Việt
- Tách từ tiếng Việt
- Chuyển chữ viết thường
- Chuẩn hóa câu

In [ ]:
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

In [ ]:
bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

In [ ]:
nguyen_am_to_ids = {}
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

Chuẩn hóa unicode

Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau 

Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn

In [ ]:
def chuan_hoa_unicode(text):
	text = unicodedata.normalize('NFC', text)
	return text

Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.


In [ ]:
"""
	Hàm này xử lý chuẩn hóa từng từ một, 
  sau khi chuẩn hóa từng từ thì sẽ chuân hóa từng câu sau 
	""" 
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

In [ ]:
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

In [ ]:
def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng

Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn 



In [ ]:
def tach_tu_tieng_viet(text):
	text = ViTokenizer.tokenize(text)
	return text

In [ ]:
# Đưa về chữ viết thường 
def chuyen_chu_thuong(text):
	return text.lower()

In [ ]:
# Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản 
def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text

In [ ]:
def tien_xu_li(text):
	text = chuan_hoa_unicode(text)
	text = chuan_hoa_dau_cau_tieng_viet(text)
	text = tach_tu_tieng_viet(text)
	text = chuyen_chu_thuong(text)
	text = chuan_hoa_cau(text)

	return text

In [ ]:
if __name__ == "__main__":
    file = open("test.txt", "r", encoding="utf-8")
    data = file.read()
    data = tien_xu_li(data)
    print("Ví dụ : \n")
    print(data)
    with open('final.txt', 'w') as f:
      f.write(data)
      f.close()

Ví dụ : 

gặp_gỡ giảng_viên đại_học fpt giành huy_chương vàng tại giải muay thế_giới vượt qua hơn 3 000 vđv đến từ 112 quốc_gia và vùng lãnh_thổ thầy nguyễn văn yên giảng_viên bộ_môn vovinam tại đại_học fpt đã xuất_sắc mang về hai tấm huy_chương danh_giá cho đội_tuyển việt_nam ngày 125 tại giải_đấu ifma senior world được đăng_cai bởi liên_đoàn hiệp_hội muaythai quốc_tế giải_đấu quy_tụ sự tham_gia tranh tài của hơn 3 000 vđv đến từ 112 quốc_gia và vùng lãnh_thổ và được đăng_cai bởi ifma liên_đoàn hiệp_hội muaythai quốc_tế từ ngày 35 1352023 trên quê_hương của môn võ này tại xứ chùa vàng thầy nguyễn văn yên giảng_viên bộ_môn voivinam tại đại_học fpt đã xuất_sắc mang về huy_chương vàng nội_dung waikru đơn nam và huy_chương đồng_nội_dung maimuay đôi nam đây là niềm vinh_dự vào tự_hào không chỉ của trường đại_học fpt mà còn là của người dân việt_nam cùng lắng_nghe những chia_sẻ của thầy yên sau khi giải_đấu kết_thúc nhé xin chúc_mừng những thành_tích xuất_sắc của thầy tại giải_đấu ifma seni

In [ ]:
#read input file
f = open("final.txt", "rt")
#read file contents to string
data = f.read()
#replace all occurrences of the required string
data = data.replace('_', ' ')
#close the input file
f.close()
#open the input file in write mode
f = open("final.txt", "wt")
#overrite the input file with the resulting data
f.write(data)
#close the file
f.close()

Remove Stopwords

In [ ]:
stopwords = open('stopword.txt', 'r')
stopwords_list = stopwords.read().split('\n')
final = open('final.txt', 'r')
final = final.read().split()
res = []
for word in final:
  if word not in stopwords_list:
    res.append(word)

res